# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


- **Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [39]:
limit=5

generator=square_root_generator(limit)

sum_of_square_roots= sum(sqrt_value for sqrt_value in generator)

print("Sum of square roots for limit =", limit, ":", sum_of_square_roots)



Sum of square roots for limit = 5 : 8.382332347441762


- **Question 2: What is the 13th number yielded**

In [41]:
limit=13

generator=square_root_generator(limit)

for _ in range(12):
    next(generator)


thirteenth_number = next(generator)

print("The 13th number yielded:", thirteenth_number)



The 13th number yielded: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [49]:
!pip install "dlt[duckdb]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 42.2 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [51]:
import dlt

# Define the generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# Initialize a DLT pipeline
pipeline = dlt.pipeline(pipeline_name="person",
                        destination='duckdb', 
                        dataset_name='person.people')  # Specify fully qualified path

# Run the pipeline with the data from the generator
info = pipeline.run(people_1(), 
                    table_name="people", 
                    write_disposition="replace")


In [63]:
pipeline = dlt.pipeline(pipeline_name="person",
                        destination='duckdb', 
                        dataset_name='person.people')
try:
    with pipeline.sql_client() as client:
        res = client.execute_sql(
            "SELECT AVG(age) FROM people"
        )
        average_age = res[0][0] 
        print(average_age)
except Exception as e:
    print("An error occurred:", e)

28.0


2. Append the second generator to the same table as the first.

In [64]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

info = pipeline.run(people_2(), 
					table_name="people", 
					write_disposition="append")

try:
    with pipeline.sql_client() as client:
        res = client.execute_sql(
            "SELECT * FROM people"
        )
        print(res)
except Exception as e:
    print("An error occurred:", e)


[(1, 'Person_1', 26, 'City_A', '1707291104.4063938', 'TT4qdc6xCa0j/A', None), (2, 'Person_2', 27, 'City_A', '1707291104.4063938', 'vL3nGH1R3Y6YsA', None), (3, 'Person_3', 28, 'City_A', '1707291104.4063938', 'EAdzRs43x7Mnig', None), (4, 'Person_4', 29, 'City_A', '1707291104.4063938', 'T9CvH1C5CHusFg', None), (5, 'Person_5', 30, 'City_A', '1707291104.4063938', 'u/V/Om964I2/uA', None), (3, 'Person_3', 33, 'City_B', '1707292378.7985077', 'lpELSLYvGgM6lA', 'Job_3'), (4, 'Person_4', 34, 'City_B', '1707292378.7985077', 'sj+9nDqrixYfJA', 'Job_4'), (5, 'Person_5', 35, 'City_B', '1707292378.7985077', 'Yd3W2XAWtaSnEw', 'Job_5'), (6, 'Person_6', 36, 'City_B', '1707292378.7985077', '5y5rcqh1eJQn1Q', 'Job_6'), (7, 'Person_7', 37, 'City_B', '1707292378.7985077', 'wXbgkQXNrj5zuw', 'Job_7'), (8, 'Person_8', 38, 'City_B', '1707292378.7985077', '1B5g1dGBZR1GAg', 'Job_8')]


In [65]:
!pip install tabulate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [72]:
from tabulate import tabulate

# Convert the result set to a list of lists and format the age column
rows = []
for row in res:
    formatted_row = list(row)
    formatted_row[2] = int(formatted_row[2])  # Convert age to integer
    # Rearrange the elements in the list to match the column order: ID, Name, Age, City, Occupation
    formatted_row = [formatted_row[0], formatted_row[1], formatted_row[2],formatted_row[3], formatted_row[4], formatted_row[5], formatted_row[6]]
    rows.append(formatted_row)

# Define column headers
headers = ["ID", "Name", "Age", "City", "dlt_load_id","_dlt_id","Occupation"]

# Print the data in a tabular format
print(tabulate(rows, headers=headers))


  ID  Name        Age  City      dlt_load_id  _dlt_id         Occupation
----  --------  -----  ------  -------------  --------------  ------------
   1  Person_1     26  City_A    1.70729e+09  TT4qdc6xCa0j/A
   2  Person_2     27  City_A    1.70729e+09  vL3nGH1R3Y6YsA
   3  Person_3     28  City_A    1.70729e+09  EAdzRs43x7Mnig
   4  Person_4     29  City_A    1.70729e+09  T9CvH1C5CHusFg
   5  Person_5     30  City_A    1.70729e+09  u/V/Om964I2/uA
   3  Person_3     33  City_B    1.70729e+09  lpELSLYvGgM6lA  Job_3
   4  Person_4     34  City_B    1.70729e+09  sj+9nDqrixYfJA  Job_4
   5  Person_5     35  City_B    1.70729e+09  Yd3W2XAWtaSnEw  Job_5
   6  Person_6     36  City_B    1.70729e+09  5y5rcqh1eJQn1Q  Job_6
   7  Person_7     37  City_B    1.70729e+09  wXbgkQXNrj5zuw  Job_7
   8  Person_8     38  City_B    1.70729e+09  1B5g1dGBZR1GAg  Job_8


3. **After correctly appending the data, calculate the sum of all ages of people.**

In [74]:
import dlt

# Create a DLT pipeline
pipeline = dlt.pipeline(pipeline_name="person",
                        destination='duckdb', 
                        dataset_name='person.people')

query = "SELECT SUM(age) AS total_age FROM people"

try:
    # Execute the SQL query within a context manager
    with pipeline.sql_client() as client:
        # Execute the SQL query
        res = client.execute_sql(query)
        
        # Fetch the result
        total_age = res[0][0]  # Access the first element of the first tuple

        # Print the total age
        print("Total age of all people:", total_age)
except Exception as e:
    print("An error occurred:", e)

Total age of all people: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [98]:
import dlt
@dlt.resource(write_disposition='merge', merge_key=('id'))

# Define the generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create a DLT pipeline
pipeline = dlt.pipeline(pipeline_name="person",
                        destination='duckdb', 
                        dataset_name='person.people')

# Load data from the first generator into the "people" table
info_1 = pipeline.run(people_1(), 
                      table_name="people",
                      write_disposition="replace")

# Merge data from the second generator into the "people" table, replacing any existing records
info_2 = pipeline.run(people_2(), 
                      table_name="people", 
                      write_disposition="merge")

# Calculate the sum of ages of all the people loaded
try:
    # Execute a SQL query to calculate the sum of ages
    with pipeline.sql_client() as client:
        query = "SELECT SUM(Age) FROM people"  # Change table name here
        result = client.execute_sql(query)
        total_age = result[0][0]
        print("Total sum of ages of all people:", total_age)
except Exception as e:
    print("An error occurred:", e)


Total sum of ages of all people: 266


In [102]:
try:
    with pipeline.sql_client() as client:
        res = client.execute_sql(
            "SELECT * FROM people"
        )
except Exception as e:
    print("An error occurred:", e)

from tabulate import tabulate

# Convert the result set to a list of lists and format the age column
rows = []
for row in res:
    formatted_row = list(row)
    formatted_row[2] = int(formatted_row[2])  # Convert age to integer
    # Rearrange the elements in the list to match the column order: ID, Name, Age, City, Occupation
    formatted_row = [formatted_row[0], formatted_row[1], formatted_row[2],formatted_row[3], formatted_row[4], formatted_row[5], formatted_row[6]]
    rows.append(formatted_row)

# Define column headers
headers = ["ID", "Name", "Age", "City", "dlt_load_id","_dlt_id","Occupation"]

# Print the data in a tabular format
print(tabulate(rows, headers=headers))


  ID  Name        Age  City      dlt_load_id  _dlt_id         Occupation
----  --------  -----  ------  -------------  --------------  ------------
   1  Person_1     26  City_A     1.7073e+09  i0ElzoDBqekyGg
   2  Person_2     27  City_A     1.7073e+09  OklctpOHEiMaNg
   3  Person_3     33  City_B     1.7073e+09  BEsBZhHm1YmjEg  Job_3
   4  Person_4     34  City_B     1.7073e+09  g/5woNTzjjc6zQ  Job_4
   5  Person_5     35  City_B     1.7073e+09  Cmpt6AKWj/3yJw  Job_5
   6  Person_6     36  City_B     1.7073e+09  mrgSO3vOyEuNRQ  Job_6
   7  Person_7     37  City_B     1.7073e+09  aLma/X87bj1TvA  Job_7
   8  Person_8     38  City_B     1.7073e+09  c6anbvp7X6XNbA  Job_8


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX